In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install pyedflib

In [ ]:
import pandas as pd
import numpy as np 
from pyedflib import highlevel
from sklearn import preprocessing
import progressbar as pb
from time import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
!ls

gdrive	sample_data


In [ ]:
import os
os.chdir('gdrive/MyDrive/')

In [ ]:
signals, signal_headers, header = highlevel.read_edf('MI_Kushagra.edf')
head = [] 
for i in range(len(signal_headers)):
    head.insert(len(head),signal_headers[i]['label'])

In [ ]:
archive=pd.DataFrame(data=signals.transpose(),columns = head)
archive['Ans'] = 0
archive['TimeStamp'] = [float(i/2048) for i in range(0,archive.shape[0])]
archive.head()

,FP1,FP2,F7,F3,FZ,F4,F8,T7,C3,CZ,C4,T8,P7,P3,PZ,P4,P8,O1,O2,A1,A2,EMG1,EMG2,ECG1,ECG2,Ans,TimeStamp
0,367.284657,1.983673,-34.637980,11.444266,-23.346304,-24.872206,31.891356,93.232624,-55.695430,-60.578317,-65.766384,-48.371099,-22.430762,-26.092927,-82.856489,-69.428550,20.599680,-40.741588,-83.772030,24.261845,-82.551308,175.631342,-490.272374,6248.416876,10000.000000,0,0.000000
1,301.060502,-29.449912,110.017548,77.668421,55.390249,22.125582,66.681926,60.883497,110.627909,-86.823835,51.728084,81.635767,173.189899,109.712367,29.144732,-24.567025,9.308003,140.230411,-28.839551,173.495079,74.616617,176.241703,-494.544900,5092.088197,10000.000000,0,0.000488
2,241.550317,-59.967956,250.095369,140.840772,131.990539,68.513008,100.251774,28.839551,271.763180,-112.153811,165.560388,207.980468,362.706950,240.634775,138.704509,18.463416,-1.373312,315.098802,24.261845,317.235065,226.901656,173.800259,-490.882734,3992.828260,10000.000000,0,0.000977
3,189.059281,-88.960098,382.543679,200.350958,204.623484,112.153811,131.380179,-1.983673,424.048219,-135.957885,273.289082,327.916381,541.237507,363.927672,242.160678,59.662776,-12.054627,479.896239,74.616617,452.735180,370.946822,170.138094,-480.201419,2968.337530,10000.000000,0,0.001465
4,142.366674,-115.510796,504.310674,254.978256,271.458000,152.742809,160.067140,-30.670634,564.431220,-157.625696,372.167544,438.696880,705.424582,477.454795,337.987335,98.115511,-21.820401,631.876097,121.309224,577.248798,503.700313,164.644846,-456.397345,2007.324331,9645.990692,0,0.001953


In [ ]:
local_data = pd.DataFrame(archive.iloc[:,[3,5,7,11,14,25,26]])
local_data.head(1000)

,F3,F4,T7,T8,PZ,Ans,TimeStamp
0,11.444266,-24.872206,93.232624,-48.371099,-82.856489,0,0.000000
1,77.668421,22.125582,60.883497,81.635767,29.144732,0,0.000488
2,140.840772,68.513008,28.839551,207.980468,138.704509,0,0.000977
3,200.350958,112.153811,-1.983673,327.916381,242.160678,0,0.001465
4,254.978256,152.742809,-30.670634,438.696880,337.987335,0,0.001953
...,...,...,...,...,...,...,...
995,67.292287,51.422904,-38.605325,126.497292,124.666209,0,0.485840
996,67.292287,51.422904,-38.605325,126.192111,124.666209,0,0.486328
997,67.292287,51.422904,-38.605325,126.192111,124.666209,0,0.486816
998,67.292287,51.117723,-38.605325,126.192111,124.666209,0,0.487305


In [ ]:
encoded = []
x = 4
for i in pb.progressbar(range(0,archive.shape[0])):
    if ((x+1<67)and(header['annotations'][x+1][0]<=float(i/2048))):
        x=x+1
    temp = header['annotations'][x][2].lower()
    if ((temp=='eyes close')or(temp=='eyes open')or(temp=='awake')):
        encoded.append(0)
    elif ((temp=="'s")or(temp=='s')):
        encoded.append(1)
    elif (temp=='b'):
        encoded.append(2)
    elif (temp=='l'):
        encoded.append(3)
    elif (temp=='r'):
        encoded.append(4)
    else:
        encoded.append(5)
local_data['Ans'] = encoded
archive['Ans'] = encoded

100% (2885632 of 2885632) |##############| Elapsed Time: 0:00:13 Time:  0:00:13


In [ ]:
local_data.head(1000)

,F3,F4,T7,T8,PZ,Ans,TimeStamp
0,11.444266,-24.872206,93.232624,-48.371099,-82.856489,0,0.000000
1,77.668421,22.125582,60.883497,81.635767,29.144732,0,0.000488
2,140.840772,68.513008,28.839551,207.980468,138.704509,0,0.000977
3,200.350958,112.153811,-1.983673,327.916381,242.160678,0,0.001465
4,254.978256,152.742809,-30.670634,438.696880,337.987335,0,0.001953
...,...,...,...,...,...,...,...
995,67.292287,51.422904,-38.605325,126.497292,124.666209,0,0.485840
996,67.292287,51.422904,-38.605325,126.192111,124.666209,0,0.486328
997,67.292287,51.422904,-38.605325,126.192111,124.666209,0,0.486816
998,67.292287,51.117723,-38.605325,126.192111,124.666209,0,0.487305


In [ ]:
print(local_data.columns[0])


F3


In [ ]:
for i in range(5):
  np1 = np.array(local_data.iloc[:,i]).reshape(-1,1)
  sc = preprocessing.MinMaxScaler()
  sc.fit(np1)
  np1 = sc.transform(np1)
  local_data[local_data.columns[i]] = np1

In [ ]:
local_data.head()

,F3,F4,T7,T8,PZ,Ans,TimeStamp
0,0.554616,0.526215,0.696196,0.530384,0.495857,0,0.000000
1,0.625163,0.599619,0.640959,0.603718,0.501457,0,0.000488
2,0.692458,0.672069,0.586243,0.674987,0.506935,0,0.000977
3,0.755852,0.740229,0.533611,0.742641,0.512108,0,0.001465
4,0.814044,0.803622,0.484627,0.805130,0.516899,0,0.001953


In [ ]:
X = np.zeros((int(local_data.shape[0]/30)-1,150,1),dtype = float)
Y = np.zeros((int(local_data.shape[0]/30)-1,1))
for i in pb.progressbar(range(int(local_data.shape[0]/30)-1)):
  X[i] = np.array(local_data.iloc[i*30:(i+1)*30,0:5].transpose()).reshape(-1,1)
  Y[i] = np.array([int(local_data.iloc[i*30:(i+1)*30,5].sum()/30)]).reshape(1)
print(X.shape,Y.shape)

100% (96186 of 96186) |##################| Elapsed Time: 0:01:40 Time:  0:01:40


(96186, 150, 1) (96186, 1)


In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(150, input_shape = (150,1),return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(50))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(6, activation = 'softmax'))
model.add(layers.Dense(1,activation = 'relu'))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error')

In [ ]:
model.build(input_shape = (100,150,1))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 150, 150)          91200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 150, 150)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 50)                40200     
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 6)                 306       
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 7         
Total params: 131,713
Trainable params: 131,713
Non-trainable params: 0
_______________________________________________

In [ ]:
model.fit(X,Y,batch_size=100,epochs = 3)

Epoch 1/3
962/962 [==============================] - 553s 574ms/step - loss: 0.5469
Epoch 2/3
962/962 [==============================] - 553s 575ms/step - loss: 0.5390
Epoch 3/3
962/962 [==============================] - 556s 578ms/step - loss: 0.5390


In [ ]:
def driver(arr):
  if (arr.shape[0]>=arr.shape[1]):
    arr = np.array(arr).reshape(1,-1,1)
    ans =  float(model.predict(arr))
    if (ans>0):
      if (ans<=1):
        return "Current Thought Process Predicted is - Neutral"
      elif (ans<=2 and ans>1):
        return "Current Thought Process Predicted is - Neutral Straight"
      elif (ans<=3 and ans>2):
        return "Current Thought Process Predicted is - left"
      elif (ans<=4 and ans>3):
        return "Current Thought Process Predicted is - right"
      elif (ans<=5 and ans>4):
        return "Current Thought Process Predicted is - Back"
      else:
        return "Current Thought Process Predicted is - Unmatched"
    else:
      "Unnatural"
  else:
    "Weird"

In [ ]:
import time
for i in np.random.randint(90000,size = 100):
  print(driver(X[i]), Y[i])
  time.sleep(1)


Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [2.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [4.]
Current Thought Process Predicted is - Neutral Straight [2.]
Current Thought Process Predicted is - Neutral Straight [5.]
Current Thought Process Predicted is - Neutral Straight [5.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process Predicted is - Neutral Straight [0.]
Current Thought Process 

array([[0.]], dtype=float32)

In [ ]:
print(model(X[1].reshape(1,-1,1)))

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
